# ▼ 크롤링 시작하기

## 1 클래스 정의

In [1]:
class MovieAPI():
    def __init__(self):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
        self.error_index = []
        self.start = 1

    # ▼ 탐색결과 append하는 함수
    def li_append(self, index_li, movie_o, mv_link, mv_image, mv_year, mv_dir, mv_actor, mv_rating):
        self.index.append(index_li)
        self.title_li.append(movie_o)
        self.link_li.append(mv_link)
        self.image_li.append(mv_image)
        self.year_li.append(mv_year)
        self.dir_li.append(mv_dir)
        self.actor_li.append(mv_actor)
        self.rating_li.append(mv_rating)

    # ▼ 리스트까지만 반환하는 함수
    def get_movie(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        self.index_list = index_list
        self.movie_list = movie_list
        self.dir_list = dir_list
        self.year_list = year_list
        self.client_id = client_id
        self.ciient_secret = client_secret
        
        import urllib.request
        import xmltodict
        import pandas as pd
        import time
        import re
        
        for i, movie_o in enumerate(movie_list):
            ### 영화명 전처리
            movie = movie_o.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
            movie = re.sub('[^가-힣A-Za-z0-9\s-]', '', movie)   # 한글, 영어, 숫자, 공백, - 빼고 지우기
            ### url 만들기
            encText = urllib.parse.quote(movie)
            params = f'"&display=100&start={self.start}'
            url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + params
            ### request 실행
            time.sleep(0.5)
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200): # 리퀘스트 정상적 #
                response_body = response.read()
                dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
                search_li = dict_data['rss']['channel']
                # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 1. 검색 결과가 하나인 경우 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                if int(search_li['total']) == 1:
                    search_li = search_li['item']
                    mv_dir = search_li['director']
                    mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                    mv_actor = search_li['actor']
                    mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                    MovieAPI.li_append(self, index_list[i], movie_o, search_li['link'], search_li['image'], search_li['pubDate'], mv_dir, mv_actor, search_li['userRating'])
                # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 1. 검색 결과가 하나인 경우 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 2. 검색 결과가 여러개인 경우 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                elif int(search_li['total']) > 1:
                    # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 검색결과 100개보다 많은지만 확인 ▼▼▼▼▼▼▼▼▼▼▼
                    if int(search_li['total']) > 100:
                        self.error_index.append([index_list[i], '검색결과 초과'])
                    # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 검색결과 100개보다 많은지만 확인 ▲▲▲▲▲▲▲▲▲▲▲  
                    # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 많든 적든 상관없이 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                    search_li = search_li['item']
                    for j in range(len(search_li)):
                        mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
                        mv_title = re.sub('[^가-힣A-Za-z0-9\s-]', '', mv_title)
                        mv_rating = search_li[j]['userRating']
                        mv_dir = search_li[j]['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li[j]['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                        mv_year = search_li[j]['pubDate']
                        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (1) 영화명 다르고, 감독 없으면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                        if mv_title.replace(' ','') != movie.replace(' ','') and mv_dir == None:
                            # 마지막이면 -------------------
                            if j == len(search_li)-1 or j == 100:
                                self.error_index.append([index_list[i], '1페이지에 없음'])
                            # 마지막 아니면 다음 j로 -------
                            continue
                        # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (1) 영화명 다르고, 감독 없으면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (2) 영화명이 다르고, 감독이 있으면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                        elif mv_title.replace(' ','') != movie.replace(' ','') and mv_dir != None:
                            # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (2)-1 마지막이면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                            if j == len(search_li)-1 or j == 100:
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명X 감독명 일치(다행) ▼▼▼▼▼▼▼▼▼▼▼
                                # ----------- append하고 탐색 중단 ----------------
                                if dir_list[i] in mv_dir: 
                                    MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                    break
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명X 감독명 일치(다행) ▲▲▲▲▲▲▲▲▲▲▲
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명X 감독명 불일치 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                                else:
                                    # 다음 페이지에 있는지, 일치하는 게 없는지 모름
                                    # 누락으로 넣어버리자 -----------------------
                                    MovieAPI.li_append(self, index_list[i], movie_o, '누락', '누락', '누락', '누락', '누락', '누락')
                                    break
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명X 감독명 불일치 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (2)-1 마지막이면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (2)-2 마지막 아니면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                            else:
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명X 감독명 일치(다행) ▼▼▼▼▼▼▼▼▼▼▼
                                if dir_list[i] in mv_dir:
                                    if year_list[i] == mv_year:
                                        # (연도까지 같으면 append하고 탐색 중단)
                                        MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                        break 
                                    else:
                                        # (연도 다르면 다음 리스트 탐색)
                                        continue
                                # ▲▲▲▲▲▲▲▲▲▲▲ 감독명 일치(다행) ▲▲▲▲▲▲▲▲▲▲▲
                                # ▼▼▼▼▼▼▼▼▼▼▼ 감독명 불일치 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                                else:
                                    # (다음거 찾아야지)
                                    continue
                                # ▲▲▲▲▲▲▲▲▲▲▲ 감독명 불일치 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (2)-2 마지막 아니면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                        # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (2) 영화명이 다르고, 감독이 있으면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (3) 영화명 같고, 감독 있으면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                        elif mv_title.replace(' ','') == movie.replace(' ','') and mv_dir != None:
                            # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (3)-1 마지막이면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                            if j == len(search_li)-1 or j == 100:
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명O 감독명 일치(다행) ▼▼▼▼▼▼▼▼▼▼▼
                                if dir_list[i] in mv_dir:
                                    MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                    break
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명O 감독명 일치(다행) ▲▲▲▲▲▲▲▲▲▲▲
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명O 감독명 불일치 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                                else:
                                    # 해당 영화로 추가된 것이 없으면 추가
                                    if self.index != [] and self.index[-1] != index_list[i]:
                                        MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                        break
                                    # 해당 영화로 추가된 것이 있으면 그냥 끝
                                    break
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명O 감독명 불일치 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (3)-1 마지막이면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (3)-2 마지막 아니면 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                            else:
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명O 감독명 일치(다행) ▼▼▼▼▼▼▼▼▼▼▼
                                # ----------- append하고 탐색 중단 ----------------
                                if dir_list[i] in mv_dir:
                                    MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                    break
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명O 감독명 일치(다행) ▲▲▲▲▲▲▲▲▲▲▲
                                # ▼▼▼▼▼▼▼▼▼▼▼ 영화명O 감독명 불일치 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                                # (중복되더라도 일단 넣고 다음 거 탐색)
                                elif dir_list[i] not in mv_dir and self.index != [] and self.index[-1] != index_list[i]:
                                    MovieAPI.li_append(self, index_list[i], movie_o, search_li[j]['link'], search_li[j]['image'], mv_year, mv_dir, mv_actor, mv_rating)
                                    continue
                                # ▲▲▲▲▲▲▲▲▲▲▲ 영화명O 감독명 불일치 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (3)-2 마지막 아니면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                        # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (3) 영화명 같고, 감독 있으면 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ (4) 그럼에도 불구하고 마지막까지 없으면 ▼▼▼▼▼▼▼▼▼
                        else:
                            if j == len(search_li)-1 or j == 100:
                                # 누락으로 넣어버리자 -----------------------
                                MovieAPI.li_append(self, index_list[i], movie_o, '누락', '누락', '누락', '누락', '누락', '누락')
                                break
                        # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ (4) 그럼에도 불구하고 마지막까지 없으면 ▲▲▲▲▲▲▲▲▲
                # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 2. 검색 결과가 여러개인 경우 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 3. 검색 안되는 경우 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
                else: 
                    self.error_index.append([index_list[i], '검색 불가'])
                    continue
                # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 3. 검색 안되는 경우 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
            
            else: # 리퀘스트 에러 #
                print("Error Code:" + rescode)

            # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 세이브포인트 만들기 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
            if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
                print(f'{index_list[i]}번째 완료')
            if i != 0 and i % 500 == 0:    # 500번마다 savepoint로 저장해두기 
                pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv(f'./files/savepoint{index_list[i]}_utf-8.csv', encoding='utf-8')
                print('savepoint', index_list[i])
            # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 세이브포인트 만들기 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
                
        # for문 끝나면 에러 리스트도 저장해주기
        pd.DataFrame(self.error_index).to_csv(f'./files/error_list_utf8.csv', encoding='utf-8')
        
        return [self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]
                
    # ▼ get_movie 실행 및 저장 함수
    def mk_movie_df(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        import pandas as pd
        
        li_to_df = MovieAPI.get_movie(self, index_list, movie_list, dir_list, year_list, client_id, client_secret)
        movie_df = pd.DataFrame(li_to_df).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '제작연도', '감독', '출연', '평점']

        # 결과 데이터 중복 제거
        df_drop_dp = movie_df.drop_duplicates(['순번'], keep='first')
        df_drop_dp

        # 인덱스 설정
        movie_df = movie_df.set_index('순번')
        df_drop_dp = df_drop_dp.set_index('순번')

        # 결과 데이터와 중복 제거 데이터 저장
        movie_df.to_csv('./files/naver_api_utf-8.csv', encoding='utf-8')
        df_drop_dp.to_csv('./files/naver_api_drop_dupl_utf-8.csv', encoding='utf-8')

        return movie_df, df_drop_dp

## 2 리스트 불러오기

### 2-1 연도 컬럼 만들기(연도 컬럼 없는 경우) 

In [3]:
import pandas as pd

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
year_df = pd.DataFrame()
for row in movie_df['개봉일']:
    year_df = pd.concat([year_df, pd.DataFrame([row.split('-')[0]])])

year_df.columns = ['연도']

year_df.index = range(len(movie_df))

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
movie_df = pd.concat([movie_df, year_df], axis=1)

movie_df.to_csv('movie_resize2_year_utf8.csv', encoding='utf-8')

### 2-2 파일 불러오기
DataFrame 불러와서 '인덱스', '영화명', '감독', '연도' 리스트로 각자 넣어주기

In [2]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize2_year_utf8.csv', encoding='utf-8', index_col=0)
display(movie_df[:6])
index_list = list(movie_df.index)
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
year_list = movie_df['연도'].to_list()

print('len =',len(index_list,), index_list[:10])
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(year_list), year_list[:10])


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,"33,121,225,810","4,163,666",사극,15세관람가,일반영화,2014
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,"31,858,660,536","3,638,287",코미디,15세관람가,일반영화,2019
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,"27,530,825,087","3,346,172",판타지,12세관람가,일반영화,2017
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,"25,842,519,330","3,233,946",드라마,12세관람가,일반영화,2014
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,"33,577,136,860","3,597,963",액션,12세관람가,일반영화,2019
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,"28,824,682,580","3,225,974",애니메이션,전체관람가,일반영화,2019


len = 4835 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len = 4835 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '괴물', '도둑들']
len = 4835 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '봉준호', '최동훈']
len = 4835 [2014, 2019, 2017, 2014, 2019, 2019, 2009, 2015, 2006, 2012]


## 3 함수 실행하기

### 3-1 변수 할당 및 실행

In [3]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI()
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df, nv_df_drop_dp = ma.mk_movie_df(index_list=index_list, movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)

display(nv_df.head())

10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료


In [26]:
movie_df[movie_df.index == 421] # 개빡치네^^...

,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
421,사랑,곽경택,롯데쇼핑㈜롯데엔터테인먼트,2007-09-19,장편,한국,0,0,2123815,0,"547,926",멜로/로맨스,15세관람가,일반영화,2007


### 3-2 누락 갯수 확인하기

In [6]:
len(movie_df) - len(nv_df)

300

### 3-3 누락 리스트 확인하기

In [9]:
import numpy as np

aa = nv_df.index.to_numpy()             # 추출 인덱스 (ndarray 1차원)
bb = np.arange(len(movie_df))          # 원본 인덱스 (ndarray 1차원)

diff_index = np.setdiff1d(bb, aa)       # 원본 인덱스(bb)에서 추출 인덱스(aa) 빼기
missed_df = movie_df.loc[diff_index]    # 원본 파일(movie_df)에서 해당 인덱스의 DataFrame 뽑아내기

missed_df.to_csv('./files/missed_utf8.csv', encoding='utf-8')
print('누락리스트',len(missed_df))
missed_df.head()

누락리스트 300


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
22,어벤져스: 인피니티 워,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2018-04-25,장편,미국,2553,99919006769,11211880,"28,631,269,069","3,008,819",액션,12세관람가,일반영화,2018
191,미이라,알렉스 커츠먼,유니버설픽쳐스인터내셔널 코리아(유),2017-06-06,장편,미국,1257,30504689978,3689325,"7,851,059,428","909,202",액션,15세관람가,일반영화,2017
212,아일랜드,마이클 베이,워너브러더스 코리아(주),2005-07-21,장편,미국,0,0,3433900,0,"1,285,400",SF,12세관람가,일반영화,2005
264,마더,봉준호,(주)씨제이이엔엠,2009-05-28,장편,한국,648,20195373838,3013523,"7,430,891,000","1,060,010",드라마,청소년관람불가,일반영화,2009
287,표적,창,(주)씨제이이엔엠,2014-04-30,장편,한국,665,22256359300,2841788,"5,348,606,900","652,750",액션,15세관람가,일반영화,2014


In [11]:
error_list = pd.read_csv('./files/error_list_utf8.csv', encoding='utf-8', index_col=0)
nofind_index_li = error_list.loc[error_list['1']=='검색 불가']['0']
nofind_df = movie_df.loc[nofind_index_li]
print('검색불가 리스트', len(nofind_df))
nofind_df.head()

검색불가 리스트 25


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
772,"10,000 BC",롤랜드 에머리히,워너브러더스 코리아(주),2008-03-13,장편,미국,359,7430607499,1140118,"2,251,461,500","326,091",어드벤처,15세관람가,일반영화,2008
1604,애프터 선셋,브렛 래트너,롯데쇼핑㈜롯데엔터테인먼트,2005-08-25,장편,미국,0,0,331595,0,"76,901",코미디,15세관람가,일반영화,2005
1732,포켓몬 더 무비 XY&Z 「볼케니온 : 기계왕국의 비밀」,유야마 쿠니히코,(주)이수C&E,2016-12-22,장편,일본,480,2147083300,282419,"425,667,500","53,872",애니메이션,전체관람가,일반영화,2016
1758,스페이스 침스:우주선을 찾아서,커크 드 미코,(주)성원아이컴,2008-07-17,장편,미국,175,1568757000,269947,"345,234,000","55,949",애니메이션,전체관람가,일반영화,2008
1759,헨젤과 그레텔: 마녀 사냥꾼 3D,토미 위르콜라,(주)씨제이이엔엠,2013-02-14,장편,미국,297,2122205602,269510,"670,928,602","80,519",액션,청소년관람불가,일반영화,2013


### 3-4 누락 리스트 다시 돌리기

In [ ]:
import pandas as pd

# 리스트 불러오기
missed_df = pd.read_csv('./files/missed_utf8.csv', encoding='utf-8', index_col=0)
display(missed_df[:6])
index_list_m = list(missed_df.index)
movie_list_m = missed_df['영화명'].to_list()
dir_list_m = missed_df['감독'].to_list()
year_list_m = missed_df['연도'].to_list()

print('len =',len(index_list_m), index_list_m[:10])
print('len =',len(movie_list_m), movie_list_m[:10])
print('len =',len(dir_list_m), dir_list_m[:10])
print('len =',len(year_list_m), year_list_m[:10])

In [ ]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI()
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df_m, nv_df_m_drop_dp = ma.mk_movie_df_m(index_list=index_list_m, movie_list=movie_list_m, dir_list=dir_list_m, year_list=year_list_m, client_id=client_id, client_secret=client_secret)

display(movie_list_m,nv_df_m)

# ▼ 아래는 함수 오류 확인용

### 0 준비하기

In [52]:
import pandas as pd

# 리스트 불러오기
missed_df = pd.read_csv('./files/missed_utf8.csv', encoding='utf-8', index_col=0)
display(missed_df[:6])
index_list_m = list(missed_df.index)
movie_list_m = missed_df['영화명'].to_list()
dir_list_m = missed_df['감독'].to_list()
year_list_m = missed_df['연도'].to_list()

print('len =',len(index_list_m), index_list_m[:10])
print('len =',len(movie_list_m), movie_list_m[:10])
print('len =',len(dir_list_m), dir_list_m[:10])
print('len =',len(year_list_m), year_list_m[:10])

,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
156,강철중: 공공의 적 1-1,강우석,(주)씨제이이엔엠,2008-06-19,장편,한국,659,28465757767,4300670,"9,048,970,100","1,298,197",드라마,15세관람가,일반영화,2008
253,하모니,강대규,(주)씨제이이엔엠,2010-01-28,장편,한국,405,21992266198,3068544,"5,451,128,000","718,993",드라마,12세관람가,일반영화,2010
329,보안관,김형주,롯데쇼핑㈜롯데엔터테인먼트,2017-05-03,장편,한국,1091,20697532509,2571491,"4,093,422,209","491,530",범죄,15세관람가,일반영화,2017
421,사랑,곽경택,롯데쇼핑㈜롯데엔터테인먼트,2007-09-19,장편,한국,0,0,2123815,0,"547,926",멜로/로맨스,15세관람가,일반영화,2007
444,소울,피트 닥터,월트디즈니컴퍼니코리아 유한책임회사,2021-01-20,장편,미국,2018,19034164180,2048137,"6,297,246,010","659,789",애니메이션,전체관람가,일반영화,2021
458,육사오(6/45),박규태,(주)홈초이스,2022-08-24,장편,한국,1159,19791166880,1980769,"3,932,407,651","385,835",코미디,12세관람가,일반영화,2022


len = 411 [156, 253, 329, 421, 444, 458, 486, 613, 672, 744]
len = 411 ['강철중: 공공의 적 1-1', '하모니', '보안관', '사랑', '소울', '육사오(6/45)', '아이들...', '더 게임', '해적: 도깨비 깃발', '생일']
len = 411 ['강우석', '강대규', '김형주', '곽경택', '피트 닥터', '박규태', '이규만', '윤인호', '김정훈', '이종언']
len = 411 [2008, 2010, 2017, 2007, 2021, 2022, 2011, 2008, 2022, 2019]


### 1 xml 형태 확인

In [68]:
movie_list_m.index('세 번째 사랑')

410

In [91]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

import urllib.request
import xmltodict
import pandas as pd
import time
import re

start_n = 404
end_n = 405

index = index_list_m[start_n:end_n]
year_list = year_list_m[start_n:end_n]
dir_list = dir_list_m[start_n:end_n]
movie_list = movie_list_m[start_n:end_n]

for i, movie_o in enumerate(movie_list):
    movie = movie_o.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
    movie = re.sub('[^가-힣A-Za-z0-9\s,-]', '', movie)   # 한글, 영어, 숫자, 공백 빼고 지우기
    encText = urllib.parse.quote(movie)
    params = f'"&display=100&start=1'
    url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + params
    time.sleep(0.5)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
        search_li = dict_data['rss']['channel']
        display(search_li)
        print(int(search_li['total']) == 1)
        if int(search_li['total']) != 0:
            # 검색 결과 있는 경우 
            search_li = search_li['item']
            if type(search_li) == list:
                # 검색 결과가 여러개인 경우
                for j in range(len(search_li)):
                    mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                    mv_title = re.sub('[^가-힣A-Za-z0-9\s,-]', '', mv_title)
                    mv_rating = search_li[j]['userRating']
                    mv_dir = search_li[j]['director']
                    mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                    mv_actor = search_li[j]['actor']
                    mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                    mv_year = search_li[j]['pubDate']
                    if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None:
                        # 영화명 같고, 평점-감독-배우 있으면,
                        if dir_list[i] in mv_dir:
                            # 평점이 0.00이 아니고, 감독명이 API에서 찾은 리스트 있으면
                            print(index_list_m[i],
                            movie_o,
                            search_li[j]['link'],
                            search_li[j]['image'],
                            mv_year,
                            mv_dir,
                            mv_actor,
                            mv_rating) 
                            print('완료')
                            break   # 뒤에 더 탐색할 필요가 없음
                        else:
                            print('다른문제1')
                            break
                    else: print('다른문제2')
            elif type(search_li) == dict:
                # 검색 결과가 하나인 경우
                mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                mv_dir = search_li['director']
                mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                mv_actor = search_li['actor']
                mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                print(index_list_m[i],
                            movie_o,
                            search_li['link'],
                            search_li['image'],
                            mv_year,
                            mv_dir,
                            mv_actor,
                            mv_rating)

{'title': "Naver Open API - movie ::'&quot;워터 릴리스&quot;'",
 'link': 'https://search.naver.com',
 'description': 'Naver Search Result',
 'lastBuildDate': 'Mon, 27 Mar 2023 18:07:21 +0900',
 'total': '0',
 'start': '1',
 'display': '0'}

False


### 2 함수 실행해서 확인

In [96]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI()
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df_m, nv_df_m_drop_dp = ma.mk_movie_df(index_list=index_list_m[404:405], movie_list=movie_list_m[404:405], dir_list=dir_list_m[404:405], year_list=year_list_m[404:405], client_id=client_id, client_secret=client_secret)

display(movie_list_m[400:412],nv_df_m)

1
1
4767번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


['백투더 씨',
 '노 맨스 랜드',
 '유아 넥스트',
 '우주로봇 씨어',
 '워터 릴리스',
 '해피 버스데이',
 '레 미제라블',
 '판타스틱 플래닛',
 '우작',
 '미싱타는 여자들',
 '세 번째 사랑']

,영화명,네이버URL,이미지URL,제작연도,감독,출연,평점
순번,,,,,,,
4767,워터 릴리스,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/06...,2007,[셀린 시아마],"[아델 에넬, 폴린 아콰르, 루이즈 블라쉬르]",8.86


In [46]:
len(nv_df_m)

58

In [47]:
ma.error_index

[[672, '검색 불가'],
 [752, '검색 불가'],
 [772, '검색 불가'],
 [1135, '검색 불가'],
 [1188, '검색 불가'],
 [1251, '검색 불가'],
 [1335, '검색 불가'],
 [1368, '검색 불가'],
 [1393, '검색 불가'],
 [1425, '검색 불가'],
 [1469, '검색 불가'],
 [1485, '검색 불가'],
 [1494, '검색 불가'],
 [1502, '검색 불가'],
 [1545, '영화명/감독명이 다름'],
 [1580, '검색 불가'],
 [1604, '검색 불가'],
 [1620, '검색 불가'],
 [1672, '영화명/감독명이 다름'],
 [1684, '검색 불가'],
 [1686, '검색 불가'],
 [1729, '검색 불가'],
 [1732, '검색 불가'],
 [1758, '검색 불가'],
 [1759, '검색 불가'],
 [1799, '검색 불가'],
 [1801, '검색 불가'],
 [1831, '검색 불가'],
 [1851, '검색 불가'],
 [1867, '검색 불가'],
 [1885, '검색 불가'],
 [1913, '검색 불가'],
 [1925, '검색 불가'],
 [1928, '검색 불가'],
 [1939, '검색 불가'],
 [1967, '검색 불가'],
 [1970, '검색 불가'],
 [2029, '검색 불가'],
 [2069, '검색 불가'],
 [2093, '검색 불가'],
 [2098, '검색 불가'],
 [2121, '검색 불가'],
 [2195, '영화명/감독명이 다름'],
 [2220, '검색 불가'],
 [2221, '검색 불가'],
 [2269, '검색 불가'],
 [2275, '검색 불가'],
 [2278, '검색 불가'],
 [2311, '검색 불가'],
 [2331, '검색 불가'],
 [2335, '검색 불가'],
 [2337, '검색 불가'],
 [2346, '검색 불가'],
 [2355, '검색 불가'],
 [2383, '검색 불

In [49]:
missed_df.loc[4821]

영화명            세 번째 사랑
감독          리처드 J. 루이스
배급사        주식회사 마운틴픽쳐스
개봉일         2012-01-12
영화형태                장편
국적                  기타
전국 스크린수             15
전국 매출액        79548000
전국 관객수           10075
서울 매출액      61,727,500
서울 관객수           7,736
장르              멜로/로맨스
등급             청소년관람불가
영화구분           독립/예술영화
연도                2012
Name: 4821, dtype: object

In [ ]:
# https://jimmy-ai.tistory.com/102
# df.drop_duplicates()

# ⊙ 맨 아래